# ARIMA forecasting

This file contains ARIMA forecasting.
The practical foundation of this code is based on:
https://github.com/Auquan/Tutorials/blob/master/Time%20Series%20Analysis%20-%202.ipynb,
https://github.com/Auquan/Tutorials/blob/master/Time%20Series%20Analysis%20-%203.ipynb.

In [1]:
import os
import sys

import pandas as pd
import numpy as np
import time

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
import statsmodels.stats as sms
from statsmodels.tsa.arima.model import ARIMA

import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
def tsplot(y, lags=None, figsize=(10, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis Plots')
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05, method='ywm')
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')        
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return

In [3]:
data = pd.read_csv('EURUSD_2019_01_03.csv', header=0, index_col='ID', parse_dates=['Datetime'])
data = data['Middle']
print(data.shape)

# log returns
# np.log(data/data.shift(1)) == np.log(data) - np.log(data.shift(1)) ≈ (data - data.shift(1)) / data.shift(1)
lrets = np.log(data/data.shift(1)).fillna(0) # np.log - is the natural logarithm
print(lrets)
print(data)

print("Data preparing done.")

(174487,)
ID
0         0.000000
1        -0.000018
2        -0.000009
3         0.000009
4         0.000009
            ...   
174482    0.000009
174483   -0.000009
174484    0.000009
174485    0.000000
174486    0.000000
Name: Ask, Length: 174487, dtype: float64
ID
0         1.13129
1         1.13127
2         1.13126
3         1.13127
4         1.13128
           ...   
174482    1.13906
174483    1.13905
174484    1.13906
174485    1.13906
174486    1.13906
Name: Ask, Length: 174487, dtype: float64
Data preparing done.


### Fitting

We must determine the order in notebook "ARIMA_fitting".

In [4]:
best_mdl = ARIMA(endog=lrets, order=(2,0,0)).fit()
best_order = (2,0,0)

C:\Users\Dom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


### Forecasting

A table with information how much events is in given time frames. It considers file "EURUSD_2019_01_03.csv".
Average:<br>
1 s ... 2 events<br>
10 s ... 20 events<br>
1 m ... 121 events<br>
5 m ... 605 events<br>
10 m ... 1210 events<br>

In [5]:
# Create a n_steps event forecast of EURUSD returns with 95%, 99% CI
n_steps = 30

fc = best_mdl.get_forecast(steps=n_steps)
fc_95 = fc.summary_frame(alpha=0.05)
fc_99 = fc.summary_frame(alpha=0.01)
print(fc_95)
print(fc_95.at[174487, 'mean'])

#fc_95 = fc.conf_int(alpha=0.05)          # only writes the confidence intervals
#print(fc.summary_frame())             # enough if we want only one confidence interval (without date index)

idx = pd.date_range(data.index[-1], periods=n_steps, freq='D') # creating index

fc_all_data = {'forecast': fc_95['mean'], 'lower_ci_95': fc_95['mean_ci_lower'], 'upper_ci_95': fc_95['mean_ci_upper'],    
           'lower_ci_99': fc_99['mean_ci_lower'], 'upper_ci_99': fc_99['mean_ci_upper']}
fc_all = pd.DataFrame(fc_all_data)  # if we create index here, DF wants to join fc_all_data to index - in fc_all_data there is no such index
fc_all.index = idx

print(fc_all)
print(lrets)
#fc_all.head()



Ask             mean   mean_se  mean_ci_lower  mean_ci_upper
174487  3.615446e-08  0.000013      -0.000026       0.000026
174488  3.634559e-08  0.000013      -0.000026       0.000026
174489  3.898825e-08  0.000013      -0.000026       0.000026
174490  3.901618e-08  0.000013      -0.000026       0.000026
174491  3.920942e-08  0.000013      -0.000026       0.000026
174492  3.921248e-08  0.000013      -0.000026       0.000026
174493  3.922661e-08  0.000013      -0.000026       0.000026
174494  3.922691e-08  0.000013      -0.000026       0.000026
174495  3.922795e-08  0.000013      -0.000026       0.000026
174496  3.922797e-08  0.000013      -0.000026       0.000026
174497  3.922805e-08  0.000013      -0.000026       0.000026
174498  3.922805e-08  0.000013      -0.000026       0.000026
174499  3.922806e-08  0.000013      -0.000026       0.000026
174500  3.922806e-08  0.000013      -0.000026       0.000026
174501  3.922806e-08  0.000013      -0.000026       0.000026
174502  3.922806e-08  0.